In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

def load_dataset():
    train_dataset = h5py.File('/content/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('/content/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

%matplotlib inline

In [ ]:
#Loading the data
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [ ]:
#Practice loading a picture
index = 37
plt.imshow(train_set_x_orig[index])
print("y = " + str(train_set_y[:,index]) + ", it's a '" + classes[np.squeeze(train_set_y[:,index])].decode("utf-8") +  "' picture.")

In [ ]:
m_train = train_set_y.shape[1] #number of training examples
m_test = test_set_y.shape[1] #number of test examples
num_px = train_set_x_orig.shape[1] #number of pixels in each image

print("Number of training examples: m_train = " + str(m_train))
print("Number of testing examples: m_test = " + str(m_test))
print("Height/Width of each image: num_px = " + str(num_px))
print("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print("train_set_x shape: " + str(train_set_x_orig.shape))
print("train_set_y shape: " + str(train_set_y.shape))
print("test_set_x shape: " + str(test_set_x_orig.shape))
print("test_set_y shape: " + str(test_set_y.shape))

In [ ]:
#convert images from each being an array to a single array represented as a column
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

In [ ]:
#standardize the dataset
train_set_x = train_set_x_flatten / 255
test_set_x = test_set_x_flatten / 255

In [ ]:
#creating sigmoid() function for model prediction
def sigmoid(x):
  y = 1/(1+np.exp(-x))
  return y

In [ ]:
#intializing model parameters
def initialize(size):
  w = np.zeros(shape=(size, 1))
  b = 0
  return w, b

In [ ]:
def propagate(w, b, X, Y):
  m = X. shape[1]

  A = sigmoid(np.dot(w.T, X) + b)
  cost = (-1/m) * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))

  dw = (1/m) * (np.dot(X, (A-Y).T))
  db = (1/m) * (np.sum(A-Y))

  grads = {"dw": dw,
          "db": db}

  assert(dw.shape == w.shape)
  assert(db.dtype == float)
  cost = np.squeeze(cost)
  assert(cost.shape == ())

  return grads, cost

In [ ]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost):
  costs = []

  for i in range(num_iterations):
    grads, cost = propagate(w, b, X, Y)
    dw = grads["dw"]
    db = grads["db"]
    w = w - dw * learning_rate
    b = b - db * learning_rate

    if i % 100 == 0:
      costs.append(cost)
    if print_cost and i % 100 == 0:
      print("Cost after iteration", i, ": ", cost)

    params = {"w": w,
              "b": b}
    grads = {"dw": dw,
             "db": b}

  return params, grads, costs

In [ ]:
def predict(w, b, X):
  m = X.shape[1]
  Y_prediction = np.zeros((1,m))
  w = w.reshape(X.shape[0], 1)

  A = sigmoid(np.dot(w.T, X) + b)

  for i in range(A.shape[1]):
    if A[0,i] > 0.5:
      Y_prediction[0,i] = 1
    else:
      Y_prediction[0,i] = 0

  assert (Y_prediction.shape == (1, m))

  return Y_prediction

In [ ]:
print("predictions = " + str(predict(w, b, X)))

In [ ]:
def model(X_train, Y_train, X_test, Y_test, num_iterations, learning_rate, print_cost):
  # initialize parameters
  w, b = initialize(X_train.shape[0])
  # implement gradient descent
  params, costs, grads = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
  #retrieve parameters
  w = params["w"]
  b = params["b"]
  #calculate
  Y_prediction_train = predict(w, b, X_train)
  Y_prediction_test = predict(w, b, X_test)
  # print results from a dictionary
  print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
  print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

  d = {"costs": costs,
        "Y_prediction_test": Y_prediction_test,
        "Y_prediction_train" : Y_prediction_train,
        "w" : w,
        "b" : b,
        "learning_rate" : learning_rate,
        "num_iterations": num_iterations}

  return d

In [ ]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)